First of all we need to download the data. We'll use the GTSRB Training Images Set as our whole Dataset and divide it later to a training and test set. In order to get this Notebook working, you need to do the following steps:
- Change the referencing folderpath to yours
- install opencv via the terminal

The Following tasks numbered in the cells (1., 2., 3., ...) are shown in the Table of Contents:

1. Importing Librarys
2. Downloading the Data
3. Renaming the Data
4. Extracting the HOG Features
5. Split the Data and Train the model and evaluating - HOG -Random Forest
6. Split the Data and Train the model and evaluating - HOG -Support Vector Machine
7. Extracting the HUE Features
8. Split the Data and Train the model and evaluating - HUE -Random Forest
9. Split the Data and Train the model and evaluating - HUE -Support Vector Machine
10. Insert your own data


In [1]:
# 1.Importing Librarys
import requests
import zipfile
import os
import numpy as np
from skimage.io import imshow, imread
from skimage.feature import hog
from skimage import exposure
import skimage.transform as transform
import skimage.io
import skimage.exposure
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.datasets import make_classification

In [1]:
# 2.Downloading the Data
# URL of the dataset
url = "https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Training_Images.zip"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Save the downloaded file
    with open("dataset.zip", "wb") as file:
        file.write(response.content)
    
    # Extract the contents of the ZIP file
    with zipfile.ZipFile("dataset.zip", "r") as zip_ref:
        zip_ref.extractall("dataset_folder_train_images")
    
    print("Dataset downloaded and extracted successfully.")
else:
    print("Failed to download the dataset.")

Dataset downloaded and extracted successfully.


In [4]:
# 3.Renaming the Data
# Renaming the datainfrastructure
def rename_folders(folder_path):
    for folder_name in os.listdir(folder_path):
        folder_old_name = os.path.join(folder_path, folder_name)
        folder_new_name = os.path.join(folder_path, str(int(folder_name)))
        
        # Rename the folder
        os.rename(folder_old_name, folder_new_name)
        
# Be sure, to insert the right folderpath here!  
rename_folders('/home/jovyan/ML Projekkkkt/App.ML/dataset_folder_train_images/GTSRB/Final_Training/Images/')

In [2]:
# 4.Extracting the HOG Features
#read the dataset and put it in an array for HOG_features:
def read_data_HOG(folder_path):
    X = []
    y = []

    for class_label in range(43):
        class_folder = os.path.join(folder_path, str(class_label))
        
        for file_name in os.listdir(class_folder):
            if file_name.endswith(".ppm"):
                file_path = os.path.join(class_folder, file_name)
                
                # Code snippet for hogz
                img = skimage.io.imread(file_path, plugin='matplotlib')
                img_gray = skimage.color.rgb2gray(img)
                reshaped_img = transform.resize(img_gray, (40, 40))
                normalized_image = skimage.exposure.rescale_intensity(reshaped_img, in_range='image', out_range=(0, 1))
                hog_features, hog_image = hog(normalized_image, orientations=8, pixels_per_cell=(6, 6), cells_per_block=(2, 2), visualize=True)
                hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 5))
                hog_features_array = hog_features.reshape(-1)
                X.append(hog_features_array)
                y.append(class_label)

    return np.array(X), np.array(y)

folder_path_train = "/home/jovyan/ML Projekkkkt/App.ML/dataset_folder_train_images/GTSRB/Final_Training/Images"

X_hog,y_hog = read_data_HOG(folder_path_train)

In [3]:
# 5.Split the Data and Train the model and evaluating - HOG - Random Forest
#random Forest for HOG_Features
X_train_forest_hog, X_test_forest_hog, y_train_forest_hog, y_test_forest_hog = train_test_split(X_hog, y_hog, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled_forest_hog = scaler.fit_transform(X_train_forest_hog)
X_test_scaled_forest_hog = scaler.transform(X_test_forest_hog)

# Create and train the Random Forest model
random_forest_hog = RandomForestClassifier()
random_forest_hog.fit(X_train_scaled_forest_hog, y_train_forest_hog)

# Make predictions on the test set
y_pred_forest_hog = random_forest_hog.predict(X_test_scaled_forest_hog)

# Evaluate the model
accuracy_forest_hog = accuracy_score(y_test_forest_hog, y_pred_forest_hog)
print("Accuracy for Random Forest:", accuracy_forest_hog)
print()

# Calculating precision, recall, and F1-score for each class
precision_forest_hog = precision_score(y_test_forest_hog, y_pred_forest_hog, average=None)
recall_forest_hog = recall_score(y_test_forest_hog, y_pred_forest_hog, average=None)
f1_forest_hog = f1_score(y_test_forest_hog, y_pred_forest_hog, average=None)

# Printing the evaluation metrics for each class
for class_label, prec, rec, f1_score in zip(np.unique(y_hog), precision_forest_hog, recall_forest_hog, f1_forest_hog):
    print(f"Class: {class_label}")
    print(f"Precision: {prec}")
    print(f"Recall: {rec}")
    print(f"F1-Score: {f1_score}")
    print()

Accuracy for Random Forest: 0.9444019382810508

Class: 0
Precision: 1.0
Recall: 0.5
F1-Score: 0.6666666666666666

Class: 1
Precision: 0.9087136929460581
Recall: 0.8830645161290323
F1-Score: 0.8957055214723926

Class: 2
Precision: 0.8550106609808102
Recall: 0.8911111111111111
F1-Score: 0.8726877040261154

Class: 3
Precision: 0.9416058394160584
Recall: 0.9214285714285714
F1-Score: 0.9314079422382672

Class: 4
Precision: 0.8950892857142857
Recall: 0.9593301435406698
F1-Score: 0.926096997690531

Class: 5
Precision: 0.7845744680851063
Recall: 0.8104395604395604
F1-Score: 0.7972972972972973

Class: 6
Precision: 0.9833333333333333
Recall: 1.0
F1-Score: 0.9915966386554621

Class: 7
Precision: 0.8960573476702509
Recall: 0.8992805755395683
F1-Score: 0.8976660682226212

Class: 8
Precision: 0.9074733096085409
Recall: 0.8471760797342193
F1-Score: 0.8762886597938144

Class: 9
Precision: 0.9779411764705882
Recall: 0.9925373134328358
F1-Score: 0.9851851851851852

Class: 10
Precision: 0.965699208443271

In [4]:
# 6.Split the Data and Train the model and evaluating - HOG - Support Vector Machine
#SVM for HOG_Features
X_train_svm_hog, X_test_svm_hog, y_train_svm_hog, y_test_svm_hog = train_test_split(X_hog, y_hog, test_size=0.2, random_state=42)

clf_hog= make_pipeline(StandardScaler(),SVC(kernel="rbf", gamma= "auto"))
clf_hog.fit(X_train_svm_hog,y_train_svm_hog)
y_pred_svm_hog = clf_hog.predict(X_test_svm_hog)
accuracy_svm_hog = accuracy_score(y_test_svm_hog, y_pred_svm_hog)
print("Accuracy for a SVM:", accuracy_svm_hog)
print()

# Calculating precision, recall, and F1-score for each class
precision_svm_hog = precision_score(y_test_svm_hog, y_pred_svm_hog, average=None)
recall_svm_hog = recall_score(y_test_svm_hog, y_pred_svm_hog, average=None)
f1_svm_hog = f1_score(y_test_svm_hog, y_pred_svm_hog, average=None)

# Printing the evaluation metrics for each class
for class_label, prec, rec, f1_score in zip(np.unique(y_hog), precision_svm_hog, recall_svm_hog, f1_svm_hog):
    print(f"Class: {class_label}")
    print(f"Precision: {prec}")
    print(f"Recall: {rec}")
    print(f"F1-Score: {f1_score}")
    print()

Accuracy for a SVM: 0.9802346340219332



TypeError: 'numpy.float64' object is not callable

In [ ]:
# 7.Extracting the HUE Features
def read_data_HUE(folder_path):
    X = []
    y = []
    
    desired_width = 29
    desired_height = 30

    for class_label in range(43):
        class_folder = os.path.join(folder_path, str(class_label))
        
        
        
        for file_name in os.listdir(class_folder):
            if file_name.endswith(".ppm"):
                file_path = os.path.join(class_folder, file_name)
                
                
            
                image = cv2.imread(file_path)
                scaled_image = cv2.resize(image, (desired_width, desired_height)) 
                image_hsv = cv2.cvtColor(scaled_image, cv2.COLOR_BGR2HSV) 
                
                border_size = 5
                image_hsv_cropped = image_hsv[border_size:-border_size, border_size:-border_size] 
                hue_values = image_hsv_cropped[:, :, 0].flatten()
                
                histogram, _ = np.histogram(hue_values, bins=256, range=[0, 256])
                normalized_histogram = histogram.astype(np.float64) / np.sum(histogram)
                dimensionality = len(normalized_histogram)
               
                
                X.append(hue_values)
                y.append(class_label)

    return np.array(X), np.array(y)

folder_path_train = "/home/jovyan/ML Projekkkkt/App.ML/dataset_folder_train_images/GTSRB/Final_Training/Images"

X_hue,y_hue = read_data_HUE(folder_path_train)

In [ ]:
# 8.Split the Data and Train the model and evaluating - HUE - Random Forest
#random Forest for HUE features
X_train_forest_hue, X_test_forest_hue, y_train_forest_hue, y_test_forest_hue = train_test_split(X_hue, y_hue, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled_forest_hue = scaler.fit_transform(X_train_forest_hue)
X_test_scaled_forest_hue = scaler.transform(X_test_forest_hue)

# Create and train the Random Forest model
random_forest_hue = RandomForestClassifier()
random_forest_hue.fit(X_train_scaled_forest_hue, y_train_forest_hue)

# Make predictions on the test set
y_pred_forest_hue = random_forest_hue.predict(X_test_scaled_forest_hue)

# Evaluate the model
accuracy_forest_hue = accuracy_score(y_test_forest_hue, y_pred_forest_hue)
print("Accuracy for Random Forest:", accuracy_forest_hue)
print()

# Calculating precision, recall, and F1-score for each class
precision_forest_hue = precision_score(y_test_forest_hue, y_pred_forest_hue, average=None)
recall_forest_hue = recall_score(y_test_forest_hue, y_pred_forest_hue, average=None)
f1_forest_hue = f1_score(y_test_forest_hue, y_pred_forest_hue, average=None)

# Printing the evaluation metrics for each class
for class_label, prec, rec, f1_score in zip(np.unique(y_hue), precision_forest_hue, recall_forest_hue, f1_forest_hue):
    print(f"Class: {class_label}")
    print(f"Precision: {prec}")
    print(f"Recall: {rec}")
    print(f"F1-Score: {f1_score}")
    print()

In [ ]:
# 9.Split the Data and Train the model and evaluating - HUE - Support Vector Machine
#SVM for HUE features
X_train_svm_hue, X_test_svm_hue, y_train_svm_hue, y_test_svm_hue = train_test_split(X_hue, y_hue, test_size=0.2, random_state=42)

clf_hue= make_pipeline(StandardScaler(),SVC(kernel="rbf", gamma= "auto"))
clf_hue.fit(X_train_svm_hue,y_train_svm_hue)
y_pred_svm_hue = clf_hue.predict(X_test_svm_hue)
accuracy_svm_hue = accuracy_score(y_test_svm_hue, y_pred_svm_hue)
print("Accuracy for a SVM:", accuracy_svm_hue)
print()

# Calculating precision, recall, and F1-score for each class
precision_svm_hue = precision_score(y_test_svm_hue, y_pred_svm_hue, average=None)
recall_svm_hue = recall_score(y_test_svm_hue, y_pred_svm_hue, average=None)
f1_svm_hue = f1_score(y_test_svm_hue, y_pred_svm_hue, average=None)

# Printing the evaluation metrics for each class
for class_label, prec, rec, f1_score in zip(np.unique(y_hue), precision_svm_hue, recall_svm_hue, f1_svm_hue):
    print(f"Class: {class_label}")
    print(f"Precision: {prec}")
    print(f"Recall: {rec}")
    print(f"F1-Score: {f1_score}")
    print()